In [94]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler,  LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import load

In [95]:
df = pd.read_csv('test.csv', sep=',')

In [96]:
df = df.replace([0, None], np.nan)
df = df.replace([-1], 0)

le = LabelEncoder()
df['region'] = le.fit_transform(df['region'])

In [97]:
df.drop(columns = ['appl_mrkr','evry_qrtr_pmnt'],axis = 1, inplace=True)

In [98]:
df[['frst_pmnt_date_y', 'frst_pmnt_date_m', 'frst_pmnt_date_d']] = df['frst_pmnt_date'].str.split('-', expand=True)

df['frst_pmnt_date_y']=df['frst_pmnt_date_y'].astype(float)
df['frst_pmnt_date_m']=df['frst_pmnt_date_m'].astype(float)
df['frst_pmnt_date_d']=df['frst_pmnt_date_d'].astype(float)

df[['lst_pmnt_date_per_qrtr_y', 'lst_pmnt_date_per_qrtr_m', 'lst_pmnt_date_per_qrtr_d']] = df['lst_pmnt_date_per_qrtr'].str.split('-', expand=True)

df['lst_pmnt_date_per_qrtr_y']=df['lst_pmnt_date_per_qrtr_y'].astype(float)
df['lst_pmnt_date_per_qrtr_m']=df['lst_pmnt_date_per_qrtr_m'].astype(float)
df['lst_pmnt_date_per_qrtr_d']=df['lst_pmnt_date_per_qrtr_d'].astype(float)

df.drop(columns = ['frst_pmnt_date','lst_pmnt_date_per_qrtr'],axis = 1, inplace=True)

In [99]:
data = {
    "year": range(2000, 2024+1),
    "vvp": [7.306,8.944,10.831,13.208,17.027,21.61,26.92,33.25,41.28,38.81,46.31,60.11,68.1,72.99,79.03,83.1,86.1,91.84,103.86,109.61,107.32,109.6,116.22,171.041,175],
    "infl": [20.8,21.5,15.8,13.7,10.9,12.7,9.7,9.0,14.1,11.6,6.8,8.4,5.0,6.7,7.8,15.5,7.0,3.7,2.9,4.5,3.4,6.7,11.9, 7.42, 4.5],
    "uroven_bezrab": [10.6, 8.9, 8.0, 8.2, 7.3, 7.2, 7.0, 6.0, 6.2, 8.2, 7.4, 6.5, 5.5, 5.5, 5.2, 5.6, 5.5, 5.2, 4.8, 4.6, 5.8, 5.6, 4.0, 3.0, 2.9 ],
    "stavka_refen": [30, 25, 21, 18, 14, 12, 11, 10, 11, 12, 8.5, 8, 8, 5, 7, 13, 10, 9, 7, 7, 4, 5, 15, 13, 16],
    "mrot": [132, 200, 450, 600, 600, 750, 1500, 2300, 3000, 4300, 4500, 4600, 5000, 5200, 5500, 5900, 6200, 7500, 10000, 11200, 12100, 12700, 14000, 16200, 19200],
    "vvp_na_dushu" : [10511, 11346, 12126, 13333, 14727, 16221, 18113, 20228, 21700, 20117, 21271, 22783, 24278, 26044, 25730, 24062, 24104, 25999, 27386, 28495, 28181, 30875, 34750, 36000, 38000],
    # "vnesh_b": [50, 60, 70, 80, 90, 100, 110, 120, 110, 100, 110, 120, 130, 140, 130, 120, 110, 120, 130, 140, 130, 140, 150, 160, 170],
    # "dolg": [7.05, 5.09, 4.744, 7.349, 7.176, 6.376, 8.154, 8.535, 5.248, -7.821, 4.503, 5.066, 4.024, -1.755, -0.736, -1.973, 0.194, 1.826, 2.807, -2.198, -2.664, 4.749, -11.91, -11],
    "kurs": [28,29,31,30,28,28,27,25,24,31,30,29,31,31,38,60,67,58,62,64,72,73,68,85,91],
}

dop_df = pd.DataFrame(data)
df = df.merge(dop_df, left_on='year', right_on='year', how="left")

In [100]:
df = df[["npo_accnts_nmbr", "pmnts_type",'clnt_cprtn_time_d','lst_pmnt_rcnc_d','oprtn_sum_per_year','pmnts_nmbr','pmnts_nmbr_per_year','mgd_accum_period','mgd_payment_period','postal_code','region','fact_addrss','frst_pmnt_date_y','lst_pmnt_date_per_qrtr_y','year','vvp','infl','uroven_bezrab','kurs','stavka_refen','mrot']]

In [101]:
X_test = df.iloc[:, :]
sc_X = sc=load('std_scaler.bin')
X_test = sc_X.transform(X_test)

In [102]:
model = CatBoostClassifier()
model.load_model('cat')
Y_test = model.predict(X_test)

In [103]:
df = pd.read_csv('test.csv', sep=',')
output_df = df[["npo_account_id", "quarter"]]
output_df['churn'] = Y_test
output_df.to_csv('output.csv', index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12356\845212728.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df['churn'] = Y_test
